<a href="https://colab.research.google.com/github/bfatmab/Sale_Forcasting_With_AI/blob/main/Sale_Forcasting_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>